# Teaching Template: Using New AI Models

- Outcomes: Learn setup, make a minimal call, understand capabilities/limits, add an evaluation, and track cost.
- Audience: Practitioners new to this model family.
- Time: ~30–60 minutes.

Prerequisites:
- Python 3.10+; optional GPU for Unsloth/HF fine-tuning.
- Environment variables: `OPENAI_API_KEY` and/or `ANTHROPIC_API_KEY` if using APIs.
- Costs: API examples may incur small charges.

> Tip: Run this notebook top-to-bottom on Colab or your environment.


## Setup
- Prints environment info
- Installs optional packages if missing
- Sets seeds for reproducibility


In [ ]:
# Pinned installs (uncomment in Colab or fresh envs)
# !pip install -q openai==1.43.0 anthropic==0.36.0 transformers==4.44.2 datasets==2.20.0


In [ ]:
import os, sys, platform, random, time
import subprocess

print('Python:', sys.version)
print('Platform:', platform.platform())

def sh(cmd):
    try:
        return subprocess.check_output(cmd, shell=True, text=True).strip()
    except Exception as e:
        return f'ERR: {e}'

print('pip openai:', sh('python -m pip show openai | sed -n "s/^Version: //p"'))
print('pip anthropic:', sh('python -m pip show anthropic | sed -n "s/^Version: //p"'))
print('pip transformers:', sh('python -m pip show transformers | sed -n "s/^Version: //p"'))
print('pip unsloth:', sh('python -m pip show unsloth | sed -n "s/^Version: //p"'))

# Optional: seed setting
import numpy as np
SEED = 42
random.seed(SEED); np.random.seed(SEED)
try:
    import torch
    torch.manual_seed(SEED)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(SEED)
        print('CUDA device:', torch.cuda.get_device_name(0))
    else:
        print('CUDA not available')
except Exception as e:
    print('Torch not installed; skipping torch seed.', e)


## Secrets & Configuration
Provide API keys via environment variables. This notebook reads them if present and skips API calls if missing.


In [ ]:
from os import getenv
OPENAI_API_KEY = getenv('OPENAI_API_KEY')
ANTHROPIC_API_KEY = getenv('ANTHROPIC_API_KEY')

def have(pkg):
    try:
        __import__(pkg)
        return True
    except Exception:
        return False

print('Have openai sdk:', have('openai'), 'Have key:', bool(OPENAI_API_KEY))
print('Have anthropic sdk:', have('anthropic'), 'Have key:', bool(ANTHROPIC_API_KEY))


## Quickstart: Minimal Calls
Below are minimal examples for OpenAI and Anthropic. These cells gracefully no-op if SDK or keys are missing.


In [ ]:
# OpenAI minimal chat completion
try:
    if have('openai') and OPENAI_API_KEY:
        import time as _t
        from openai import OpenAI
        client = OpenAI(api_key=OPENAI_API_KEY)
        t0 = _t.time()
        resp = client.chat.completions.create(
            model='gpt-4o-mini',
            messages=[{'role':'user','content':'Say hello in 5 words.'}],
            temperature=0
        )
        print(resp.choices[0].message.content)
        usage = resp.usage
        if usage:
            print('Tokens total:', usage.total_tokens)
        print(f'Latency: {dt:.2f}s')
    else:
        print('OpenAI SDK or key missing; skipping.')
except Exception as e:
    print('OpenAI call error:', e)


In [ ]:
# Anthropic minimal message
try:
    if have('anthropic') and ANTHROPIC_API_KEY:
        import anthropic
        ac = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)
        msg = ac.messages.create(
            model='claude-3-5-sonnet-latest',
            max_tokens=64,
            messages=[{'role':'user','content':'Say hello in 5 words.'}]
        )
        print(msg.content[0].text)
    else:
        print('Anthropic SDK or key missing; skipping.')
except Exception as e:
    print('Anthropic call error:', e)


## Model Capabilities & Limits
Briefly summarize what the model is good at, known constraints (context, rate limits), and recommended parameters.


## Guided Steps
1. Define the task
2. Design the prompt or schema
3. Add a tool (if needed)
4. Run a small batch
5. Evaluate results and iterate


## Release Notes & License
Key details to know before you ship or evaluate.


In [ ]:
try:
    from huggingface_hub import HfApi
    import re, requests
    # Requires HF_MODEL like 'org/name' to be defined in your notebook context
    HF_MODEL = globals().get('HF_MODEL', 'org/name')
    api=HfApi()
    info=api.model_info(HF_MODEL)
    print('Last modified:', getattr(info,'lastModified', None))
    print('License:', getattr(info,'license', None))
    url=f'https://huggingface.co/{HF_MODEL}/raw/main/README.md'
    r=requests.get(url,timeout=10)
    if r.status_code==200:
        md=r.text
        def extract(section):
            m=re.search(r'(^|\n)#+\s*'+re.escape(section)+r'[^\n]*\n(.+?)(\n#+|\Z)', md, re.S|re.I)
            return (m.group(2).strip() if m else None)
        for sec in ['Intended Use','Use cases','Limitations','Risks','Training data','Model details','License']:
            txt=extract(sec)
            if txt:
                print(f'--- {sec} ---')
                print('\n'.join(txt.splitlines()[:20]))
except Exception as e:
    print('Could not load release notes/license details:', e)


### License Snippet & What It Means
Pulls license file text when available and summarizes practical implications.


In [ ]:
try:
    from huggingface_hub import HfApi, hf_hub_download
    import os
    HF_MODEL = globals().get('HF_MODEL', 'org/name')
    api=HfApi()
    info=api.model_info(HF_MODEL)
    lic=getattr(info,'license',None)
    print('License identifier:', lic)
    path=None
    for name in ['LICENSE','LICENSE.txt','LICENSE.md','LICENSE.MD','license','LICENSE.rst']:
        try:
            path=hf_hub_download(repo_id=HF_MODEL, filename=name, revision='main')
            break
        except Exception:
            pass
    if path and os.path.exists(path):
        text=open(path,'r',encoding='utf-8',errors='ignore').read()
        print('\n--- License snippet (first 1500 chars) ---\n')
        print(text[:1500])
    else:
        print('License file not found; see model card.')
    summary={
      'mit':'Permissive: commercial use allowed, attribution and license notice required; no warranty.',
      'apache-2.0':'Permissive with patent grant: commercial use allowed; keep NOTICE; mind patents/trademarks.',
      'bsd-3-clause':'Permissive: commercial use allowed; attribution required; no endorsement.',
      'gpl-3.0':'Copyleft: derivatives must be GPL; not suitable for closed-source distribution.',
      'lgpl-3.0':'Weak copyleft: dynamic linking OK; modified library must remain LGPL.',
      'agpl-3.0':'Network copyleft: providing as a service triggers source-sharing obligations.',
      'cc-by-4.0':'Attribution required; commercial use allowed; keep notices.',
      'cc-by-nc-4.0':'Non-commercial: no commercial use permitted; attribution required.',
      'openrail':'Responsible AI License: usage restrictions apply; review terms for safety/commercial limits.',
      'openrail++':'Responsible AI License: stricter usage constraints; check allowed and disallowed uses.',
    }
    key=(lic or '').lower()
    explain=None
    for k,v in summary.items():
        if k in key:
            explain=v; break
    print('\n--- What this license means for you ---\n')
    if explain:
        print(explain)
    else:
        print('Review the license in the model card. Typical items: attribution, redistribution terms, commercial-use scope, and safety constraints.')
except Exception as e:
    print('License lookup error:', e)


### Intended Use & Limitations (from Model Card)
Quickly scan the most relevant sections from the README.


In [ ]:
import re, requests
HF_MODEL = globals().get('HF_MODEL', 'org/name')
def fetch_readme(org_model: str, timeout: int = 10):
    url = f'https://huggingface.co/{org_model}/raw/main/README.md'
    try:
        r = requests.get(url, timeout=timeout)
        if r.status_code == 200:
            return r.text
    except Exception as e:
        print('Fetch error:', e)
    return None
def extract_sections(md_text: str, sections):
    out = {}
    for sec in sections:
        m = re.search(r'(^|\n)#+\s*' + re.escape(sec) + r'[^\n]*\n(.+?)(\n#+|\Z)', md_text, re.S | re.I)
        if m:
            out[sec] = m.group(2).strip()
    return out
md = fetch_readme(HF_MODEL)
secs = extract_sections(md or '', ['Intended Use','Use cases','Limitations','Risks','Training data'])
for k,v in secs.items():
    print(f'\n--- {k} ---\n')
    print('\n'.join(v.splitlines()[:30]))


### Knowledge Check — License & Intended Use


In [ ]:
q1='Which license allows commercial use with attribution and notice?'
opts1=['CC-BY-NC-4.0','MIT','AGPL-3.0','GPL-3.0']
ans1=1
q2='If the model card lists "Limitations" that include factual drift, what should you do?'
opts2=['Ignore and deploy','Add guardrails/evals and consider RAG','Increase temperature','Hide errors from users']
ans2=1
print(q1)
for i,o in enumerate(opts1): print(f'  {i}) {o}')
try:
    c1=int(input('Your choice (0-3): ').strip() or -1)
except Exception:
    c1=-1
print('Correct!' if c1==ans1 else 'Review license summaries above.')
print(); print(q2)
for i,o in enumerate(opts2): print(f'  {i}) {o}')
try:
    c2=int(input('Your choice (0-3): ').strip() or -1)
except Exception:
    c2=-1
print('Correct!' if c2==ans2 else 'Check Intended Use & Limitations.')


In [ ]:
# Example: structured output validation with pydantic (optional)
try:
    from pydantic import BaseModel
    class Item(BaseModel):
        title: str
        rating: int
    print('Pydantic available; define schemas for JSON outputs.')
except Exception:
    print('Pydantic not installed; skip schema validation example.')


## Evaluation
Add a tiny golden set or a simple acceptance test to verify success.


In [ ]:
# Simple acceptance test example
def is_hello_five_words(text: str) -> bool:
    return len(text.split()) == 5

print('Example check:', is_hello_five_words('Hello from a friendly bot'))


In [ ]:
# Minimal deterministic evaluation (assert)
golden = ['Hello from a friendly bot']
ok = sum(int(is_hello_five_words(s)) for s in golden)
assert ok == 1, 'Eval check failed'
print('Accuracy: 100% (1/1)')


## Cost & Observability
Show how to access token usage/cost where supported.


In [ ]:
# Example: OpenAI usage (if available on the response)
# See quickstart cell above for usage fields.
pass


In [ ]:
# Example: Anthropic admin usage/cost API (requires proper entitlements)
print('Refer to Anthropic usage & cost API notebooks for end-to-end examples.')


## Troubleshooting
- 401/403: check API keys and org permissions
- 429: add exponential backoff; reduce concurrency; shorten prompts
- JSON errors: validate with schemas; enable JSON/structured modes if available
- Timeouts: reduce payloads; increase timeout; retry idempotently


### Diagnostics (Copy/Paste Ready)

In [ ]:
# Connectivity check for OpenAI-compatible servers (e.g., GPT-OSS via Ollama/vLLM)
try:
    import os, requests
    base = os.getenv('OPENAI_BASE_URL', 'http://localhost:11434/v1')
    r = requests.get(base + '/models', timeout=5)
    print(r.status_code, r.text[:200])
except Exception as e:
    print('Conn error:', e)


In [ ]:
# GPU memory snapshot (if torch available)
try:
    import torch
    if torch.cuda.is_available():
        free, total = torch.cuda.mem_get_info()
        print('VRAM (GiB):', round(free/2**30,2), '/', round(total/2**30,2))
    else:
        print('CUDA not available')
except Exception as e:
    print('Torch not installed or mem_get_info unsupported:', e)


In [ ]:
# Retry/backoff helper
import time
def with_retry(fn, tries=3, backoff=1.5):
    for i in range(tries):
        try:
            return fn()
        except Exception as e:
            print(f'Attempt {i+1} error:', e)
            if i == tries-1: raise
            time.sleep(backoff**i)


In [ ]:
# JSON repair example
import json, re
def try_json(text):
    try:
        return json.loads(text), None
    except Exception as e:
        cleaned = re.sub(r'```(json)?|```', '', text).strip()
        cleaned = re.sub(r'[^\{\}\[\]\:,\"\-0-9a-zA-Z\s]', '', cleaned)
        try:
            return json.loads(cleaned), None
        except Exception as ee:
            return None, (e, ee)


## Exercises
- Change the prompt to require a JSON schema and validate it
- Add a tool/function call and parse its output
- Create a 20-item golden set and compute accuracy
- Measure average latency/tokens over 10 runs


## Cleanup
- Stop any background jobs or servers
- Clear temp files or caches if created
- Save artifacts/models if you trained locally
